In [ ]:
import time

import geopandas as gpd
tiles = gpd.read_parquet('example_data/fabdem_tiles.parquet')
russia = gpd.read_parquet('example_data/russia_regions.parquet')


In [ ]:
tiles_russia = tiles.sjoin(russia,how='inner').drop_duplicates(subset='tile_name')
tiles_russia[tiles.columns].to_parquet('example_data/fabdem_tiles_russia.parquet')

In [ ]:
tiles_russia

In [ ]:
tiles_russia.iloc[1000:1006].explore()

In [ ]:
from tqdm import tqdm
from pathlib import Path
import time
import geopandas as gpd
import requests


OUTPUT_ROOT = Path("soilgrid_tiles")

VARS = ["clay", "sand", "silt", "cfvo"]
DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]

WCS_URLS = {
    "clay": "https://maps.isric.org/mapserv?map=/map/clay.map",
    "sand": "https://maps.isric.org/mapserv?map=/map/sand.map",
    "silt": "https://maps.isric.org/mapserv?map=/map/silt.map",
    "cfvo": "https://maps.isric.org/mapserv?map=/map/cfvo.map",
}
WCS_VERSION = "2.0.1"
TILE_NAME_FIELD = "tile_name"


def download_soilgrids_tile(var_name, depth, bounds, out_path):

    minx, miny, maxx, maxy = bounds

    wcs_url = WCS_URLS[var_name]
    coverage_id = f"{var_name}_{depth}_mean"

    params = {
        "SERVICE": "WCS",
        "VERSION": WCS_VERSION,
        "REQUEST": "GetCoverage",
        "coverageId": coverage_id,
        "SUBSETTINGCRS": "EPSG:4326",
        "OUTPUTCRS": "EPSG:4326",
        "format": "image/tiff",
        "subset": [
            f"Lat({miny},{maxy})",
            f"Long({minx},{maxx})",
        ],
    }

    params_list = [
        ("SERVICE", params["SERVICE"]),
        ("VERSION", params["VERSION"]),
        ("REQUEST", params["REQUEST"]),
        ("coverageId", params["coverageId"]),
        ("SUBSETTINGCRS", params["SUBSETTINGCRS"]),
        ("OUTPUTCRS", params["OUTPUTCRS"]),
        ("format", params["format"]),
        ("subset", params["subset"][0]),
        ("subset", params["subset"][1]),
    ]

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    r = requests.get(wcs_url, params=params_list, stream=True, timeout=300)

    if not r.ok:
        raise RuntimeError(
            f"WCS запрос не удался ({r.status_code}): {r.text[:500]}"
        )

    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)



gdf = gpd.read_parquet('example_data/fabdem_tiles_russia.parquet')


if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(4326)


for idx, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Тайлы"):
    tile_name = str(row[TILE_NAME_FIELD])

    geom = row.geometry
    if geom is None or geom.is_empty:
        print(f"Пропускаю тайл {tile_name}: пустая геометрия")
        continue

    minx, miny, maxx, maxy = geom.bounds

    tile_root = OUTPUT_ROOT / tile_name

    for var in VARS:
        for depth in DEPTHS:
            filename = f"{var}_{depth}_mean.tif"
            out_path = tile_root / var / filename

            if out_path.exists():
                time.sleep(0.0001)
                continue

            try:
                download_soilgrids_tile(
                    var_name=var,
                    depth=depth,
                    bounds=(minx, miny, maxx, maxy),
                    out_path=out_path,
                )
            except Exception as e:
                print(f"Ошибка для {tile_name} {var} {depth}: {e}")
